# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @dudabicalhocd

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [5]:
#Produto escolhido:
produto = 'Bradesco'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 400

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [8]:
print(os.getcwd())
dados = pd.read_excel('Bradesco.xlsx')

C:\Users\milen\OneDrive\Documentos\2o SEMESTRE\CDD\Projeto2-CD


In [9]:
#pegando os dados interessantes para nos, Relevantes
dr= dados.Respostas[dados.Relevancia=='r']
print(len(dr))
probder= 91/300

91


In [10]:
#Irrelevantes
di= dados.Respostas[dados.Relevancia=='i']
print(len(di))
probdei=209/300

209


In [14]:
dicior={} #dicionario que contem a palavra e a quantidade de vezes que ela aparece.
palavras=[]
Desconsiderar=0 #palavras que podemos desconsiderar, como sites
# limpando as palavras e as adicionando a um dicionario
for x in dr:
    nomes=x.split()
    for y in nomes:
        p2=[]
        if y[0]=='h' and y[1]=='t':
            y='.'
            Desconsiderar+=1
        if y[0]== '@':
            y='.'
            Desconsiderar+=1
        for i in y : 
            if i!= ":" and i!="'" and i!='"' and i!=' ' and i!= "," and i!= "." and i!='?'and i!='!' and i!='(' and i!=')' :
                p2.append(i)
        a="".join(p2)
        palavras.append(a)
        if x in dicior:
            dicior[y]+=1
        else:
            dicior[y]=1
        
            
#print(dicior)
#print(palavras)

#total de palavras relevantes
tr=sum(dicior.values())-Desconsiderar
print(tr)     

606


In [15]:
dicioi={} #dicionario que contem a palavra e a quantidade de vezes que ela aparece.
palavrasi=[]
Desconsiderari=0 #palavras que podemos desconsiderar, como sites
# limpando as palavras e as adicionando a um dicionario
for x in di:
    nomes=x.split()
    for y in nomes:
        p2=[]
        if y[0]=='h' and y[1]=='t':
            y='.'
            Desconsiderari+=1
        if y[0]== '@':
            y='.'
            Desconsiderari+=1  
        for i in y:
            if i!= ":" and i!="'" and i!='"' and i!=' ' and i!= "," and i!= "." and i!='?'and i!='!' and i!='(' and i!=')' and i!='…' :
                p2.append(i)
        a="".join(p2)
        palavrasi.append(a)
        if y in dicioi:
            dicioi[y]+=1
        else:
            dicioi[y]=1

#print(dicioi)
#print(palavrasi)

#total de palavras irrelevantes
ti=sum(dicioi.values())-Desconsiderari
print(ti)

2687


In [16]:
#total de palavras existentes nos tweets
soma=len(dicioi)
for i in dicior:
    if i in dicioi:
        soma+=0
    else:
        soma+=1

t=soma 
print(t)

1295


In [22]:
#probabilidade da frase ser relevante

frase="muito ruim" #frase de exemplo
def probabr(frase,dicior,tr,t):
    nomes=frase.split()
    pfr=1
    for i in nomes:
        if i in dicior:
            pa=(dicior[i]+1)/(tr+t)
            pfr*=pa
        else:
            pa=1/(tr+t)
            pfr*=pa
    return pfr
    
print(probabr(frase,dicior,tr,t))

5.534339051873636e-07


In [23]:
#probabilidade da frase ser irrelevante

frase="muito ruim"
def probabi(frase,dicioi,ti,t):
    nomes=frase.split()
    pfi=1
    for i in nomes:
        if i in dicioi:
            pa= (dicioi[i]+1)/(ti+t)
            pfi*=pa
        else:
            pa=1/(ti+t)
            pfi*=pa
    return pfi
print(probabi(frase,dicioi,ti,t))

1.2613263957018036e-07


In [24]:
#comparando qual probabilidade é maior, para vermos se a frase é ou não releante
if probabi(frase,dicioi,ti,t)<probabr(frase,dicior,tr,t):
    print('É relevante')
else:
    print('É irrelevante')

É relevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [25]:
print(os.getcwd())
dados2 = pd.read_excel('Bradesco.xlsx', 1)

C:\Users\milen\OneDrive\Documentos\2o SEMESTRE\CDD\Projeto2-CD


In [26]:
#pegando os dados para a analise
dados2.head()
d2= dados2.Tweets
dad2=dados2.Relevancia
print(d2)

0          só o bradesco que n vai ter feriadão, q bosta
1      novidades bradesco saúde. https://t.co/ndk1na7vag
2      são 84 cursos grátis e com certificado da fund...
3      @bradesco cheguei 12:05h agência cachambi em f...
4      rt @bradesco: a história de orgulho dos gaúcho...
5      eu oficialmente detesto o bradesco e essa agên...
6      humilhada na chuva na porta do bradesco com um...
7      @marcosrebello01 olá! por favor, nos passe ess...
8      rt @bradesco: a história de orgulho dos gaúcho...
9      meninas que confundem santander com bradesco s...
10     vontade de tomar um chima pqp https://t.co/5g3...
11     o banco do br ia voltar essa semana, aí explod...
12     rt @marcelomassaobr: i'm at banco bradesco in ...
13     salgueiro-pe: bandidos sequestram gerente admi...
14     rt @bradesco: a história de orgulho dos gaúcho...
15                      @oiayres olá! o que foi cobrado?
16     rt @bradesco: a história de orgulho dos gaúcho...
17     @bradesco bloquear e ped

In [34]:
Relevancia2=[]
ii=0
ir=0
rr=0
ri=0

#definindo a relevancia de cada frase da base Teste calculada pelo nosso código
for i in d2:
    pi= probabi(i,dicioi,ti,t)
    pr= probabr(i,dicior,tr,t)
    if pi<pr:
        Relevancia2.append('r')
    else:
        print(i)
        Relevancia2.append('i')
        
#fazendo uma lista de relevancia marcada por nós da base Teste
Relevancia=[]
for i in dad2:
    Relevancia.append(i)
    
#comparando os resultados    
for x in range(len(Relevancia2)):
    if Relevancia[x]=='i' and Relevancia2[x]=='i':
        ii+=1
    if Relevancia[x]=='i' and Relevancia2[x]=='r':
        ir+=1
    if Relevancia[x]=='r' and Relevancia2[x]=='r':
        rr+=1
    if Relevancia[x]=='r' and Relevancia2[x]=='i':
        ri+=1
            

print(ii)
print(ir)
print(rr)
print(ri)

print('-----')
print(ii+ir+rr+ri)



só o bradesco que n vai ter feriadão, q bosta
novidades bradesco saúde. https://t.co/ndk1na7vag
são 84 cursos grátis e com certificado da fundação bradesco, aproveite para dar um up em sua carreira!
marque... https://t.co/4hgfb79rsk
rt @bradesco: a história de orgulho dos gaúchos vira festa em setembro. a semana farroupilha tá chegando! vem conhecer mais: https://t.co/h…
eu oficialmente detesto o bradesco e essa agência 👌👌
humilhada na chuva na porta do bradesco com uma mochila de 30 kg
@marcosrebello01 olá! por favor, nos passe essas informações e envie mais detalhes para o e-mail 4165.patrocinios@bradesco.com.br. :)
rt @bradesco: a história de orgulho dos gaúchos vira festa em setembro. a semana farroupilha tá chegando! vem conhecer mais: https://t.co/h…
meninas que confundem santander com bradesco só pq é tudo vermelho me add
o banco do br ia voltar essa semana, aí explodem o banco do bradesco, oh povo sem sorte
rt @marcelomassaobr: i'm at banco bradesco in são paulo https://t.co/zw

In [28]:
#Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
pri=ri/45 *100
print('pri =')
print(pri)
#Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
prr=rr/45*100
print('prr =')
print(prr)
#Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
pii=ii/155*100
print('pii =')
print(pii)
#Porcentagem de negativos falsos (marcado como não relevante e são relevantes)
pir=ir/155*100
print('pir =')
print(pir)

pri =
93.33333333333333
prr =
6.666666666666667
pii =
88.38709677419355
pir =
11.612903225806452


___
## Concluindo

Escreva sua conclusão aqui.<br />
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


<br />
Podemos concluir que nosso Classificador pode analisar bem os dados considerados irrelevantes, apresentando uma porcentagem de aproximadamente 88,4% de acertos, mas que sua analise sobre os relevantes não foi tão boa, com apenas 6% de acertos. <br />
<br />

Podemos perceber que algumas frases consideradas relevantes pelo programa mas não por nós, acabam se tornando relevantes por apresentar palavras mais difíceis de nosso sistema reconhecer pela filtragem que fizemos nas palavras, explicando a ineficácia do programa. Alguns exemplos são:<br />
-@robertolasilva @glaysinha @john_silva2 @anacec97 @xarope_bolado @josi_sborges @matheuscecmdm @daabragahttps://t.co/08emukfgyp<br />
-@stevenstrogatz @jsellenberg @luisc @bancopaulista @bradesco @mobil @votorantim @citibank @billgates @batinaphttps://t.co/py2txblpse<br />
-#timbeta #betaajudabeta #timbetaajudatimbeta #betaajudabeta  #betaquerlab  #timrepin #pinterest (@ bradesco)https://t.co/m0x0tscsy9<br />
<br />
<br />
Já, podemos perceber que as palavras que por nós eram relevantes, mas consideradas irrelevantes pelo sistema, foram consideradas assim pela quantidade de palavras importantes ser grande e o dicionário que usamos como base para as irrelevantes terem maior número de palavras, por grande parte dos tweets da base de Treinamento ser considerada Irrelevante,209 de 300 frases, contando com 2687 palavras irrelevantes, e 91 de 300 frases eram relevantes, com 606 palavras diferentes. Alguns exeplos de frases relevantes consideradas irrelevantes são:<br />
-@bradesco oi isa bom dia. cá estou na agencia do bradesco e apenas 2 cx funcionando em época de pgto de idosos e sehttps://t.co/bbvr6jsccx<br />
-@bradesco tô com dificuldade de tirar o dinheiro da minha conta, me ajuda<br />
<br />
<br />
(não sei o que falar das palavras de sarcasmo!!)<br />
<br />
Essa analise evidencia  a importância de uma melhora no projeto, contando com mais características para poder distinguir frases relevantes e irrelevantes, e para isso um maior investimento nele. Isso ajuda no desenvolvimento e melhoria do Banco, que consegue analisar diretamente as frases relevantes para seu estudo, e assim poder saber com mais facilidade os problemas enfrentados por seus  usuários.
<br />